# Module Refinement Example

In [ ]:
# Load necessary packages
import numpy as np
import pandas as pd
import os
from ModuleRefinement import utils, ModuleLBG, center_algorithms
import plotly.express as px

from sklearn.model_selection import StratifiedKFold

from orthrus.core.dataset import DataSet as DS

from IPython.utils import io

In [2]:
# choose the module representative
center_method = 'flag_mean'
data_dimension = 1
center_dimension = 1

# options: 
#     center_method:
#         eigengene
#             data_dimension:
#                 1
#             center dimension:
#                 1,2,3,...,n
#         flag_mean
#             data_dimension:
#                 1,2,3,...,n
#             center dimension:
#                 1,2,3,...,n
#         flag_median
#             data_dimension:
#                 1,2,3,...,n
#             center dimension:
#                 1,2,3,...,n
#         module_expression
#             data_dimension
#                 1
#             center dimension
#                 1

'\noptions: \n    center_method:\n        eigengene\n            data_dimension:\n                1\n            center dimension:\n                1,2,3,...,n\n        flag_mean\n            data_dimension:\n                1,2,3,...,n\n            center dimension:\n                1,2,3,...,n\n        flag_median\n            data_dimension:\n                1,2,3,...,n\n            center dimension:\n                1,2,3,...,n\n        module_expression\n            data_dimension\n                1\n            center dimension\n                1\n'

In [3]:
#name of the dataset: 'salmonella_Liver' or 'gse73072_hrv_48_64'
dataset_name = 'salmonella_Liver' 
#which fold number (0, 1, 2, 3, or 4) we want to run on.
fold = 1

#path to modules file
out_dir = './modules/5fold'
out_file = os.path.join(out_dir, f'fold{fold}_{dataset_name}.pickle')
module_file = out_file

#path to refined modules file
refined_modules_dir = './refined_modules'

if not os.path.exists(out_dir):
    os.mkdir(out_dir)
if not os.path.exists(refined_modules_dir):
    os.mkdir(refined_modules_dir)

In [ ]:
#a list of lists of center methods
center_methods = [[center_method,data_dimension,center_dimension]]

#paths to modules and refined modules
refined_modules_file = f'{refined_modules_dir}/{center_method}_{data_dimension}_{center_dimension}/5fold/fold{fold}_{dataset_name}.pickle'
module_paths = [module_file,
                refined_modules_file]

#animal species for the dataset ('human' for gse73072 dataset, 'mouse' for salmonella dataset)
if dataset_name == 'gse73072_hrv_48_64':
    species = 'mouse' 
    organism = 'hsapiens' 
elif dataset_name == 'salmonella_Liver':
    species = 'mouse'
    organism = 'mmusculus' 
else:
    print(f'dataset_name: {dataset_name} not recognized!')

#two algorthm names (no need to change!)
algorithms = ['WGCNA', 'LBG']

#path to the dataset
data_path = f'../data/{dataset_name}.csv'

In [ ]:
# Load data
class_data, unique_labels, data_all, labels_all = utils.load_data(data_path)

## Compute WGCNA Modules 

*Takes approximately 10 minutes*

In [ ]:
 #keep random state at 0 for reproducible results
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
skf.get_n_splits(data_all, labels_all)
fold_number = 0 
for train_index, test_index in skf.split(data_all, labels_all):
    if fold_number == fold:
        split_data = data_all.iloc[train_index]
        split_labels = labels_all.iloc[train_index]

        with io.capture_output() as captured:
            r_value = utils.wgcna_modules(split_data, species, out_file)

        print(f'MODULES COMPUTED! R value = {r_value}')
    fold_number +=1

## Compute the Refined Modules

In [ ]:
utils.refined_modules(split_data, module_file, center_methods, refined_modules_dir)

## Compute GO Significance
Store in `go_results`.

In [ ]:
go_results = pd.DataFrame(columns = 
                                    ['Data Set', 'Algorithm', 'Central Prototype', 
                                    'Data Dimension', 'Center Dimension', 'Fold', 
                                    'Module Number', 'GO Significance'])


for module_path, algorithm in zip(module_paths, algorithms):
    
    print(f'GO for {algorithm} modules')

    the_modules, all_features = utils.load_modules(module_path)
    module_number = 0
    for module in the_modules.iterrows():
        module_genes = module[1].item()   
                        
        mod_sig = utils.module_significance(module_genes, organism)

        if algorithm == 'WGCNA':
            row = pd.DataFrame(columns = list(go_results.columns),
                    data = [[dataset_name, algorithm, '',
                                '', '', fold,
                                module_number, mod_sig]])
        else:
            row = pd.DataFrame(columns = list(go_results.columns),
                    data = [[dataset_name, algorithm, center_method,
                                data_dimension, center_dimension, fold,
                                module_number, mod_sig]])            


        go_results = pd.concat([go_results, row])
        
        module_number +=1

go_results.head()

## Compute the relative gain in GO significance
Store in `go_better`

In [ ]:
go_better = pd.DataFrame(columns = ['Relative Gain', 'Fold', 'Prototype', 'Data Dimension', 'Center Dimension', 'Data Set'])

wgcna_query = (go_results["Algorithm"] == 'WGCNA') &\
              (go_results["Fold"] == fold) &\
              (go_results["Data Set"] == dataset_name)
if np.sum(wgcna_query) > 0:
    wgcna_total = go_results[wgcna_query]["GO Significance"].sum()
    eg_query = (go_results["Algorithm"] == 'LBG') &\
            (go_results["Central Prototype"] == center_method) &\
            (go_results["Data Dimension"] == data_dimension) &\
            (go_results["Center Dimension"] == center_dimension) &\
            (go_results["Fold"] == fold) &\
            (go_results["Data Set"] == dataset_name)                           
    eigengene_total = go_results[eg_query]["GO Significance"].sum()
    if np.sum(eg_query) > 0:
        relative_gain = (eigengene_total/(wgcna_total+.0001))-1
        row = pd.DataFrame(columns = go_better.columns,
                        data = [[relative_gain, fold, center_method, data_dimension, center_dimension, dataset_name]])
        go_better = go_better.append(row, ignore_index = True)
go_better

## SVM BSR for each module. 
Store in `svm_results`

In [ ]:
ds = DS(data = data_all, metadata = labels_all)

svm_results = pd.DataFrame(columns = 
                                ['Data Set', 'Algorithm', 'Central Prototype', 
                                'Data Dimension', 'Center Dimension', 'Fold', 
                                'Module Number', 'BSR', 'Size'])

for module_path, algorithm in zip(module_paths, algorithms):
    
    print(f'SVM for {algorithm} modules')

    the_modules, all_features = utils.load_modules(module_path)

    module_number = 0
    for module in the_modules.iterrows():
        mod_sig = 0
        module_genes = module[1].item()   

        module_size = len(list(module_genes))

        slice_dataset = ds.slice_dataset(feature_ids=np.array(module_genes))

        try:
            bsr = utils.loso_test(slice_dataset, fold)
        except ValueError:
            bsr = np.nan   

        if algorithm == 'WGCNA':
            row = pd.DataFrame(columns = list(svm_results.columns),
                        data = [[dataset_name, algorithm, np.nan,
                                    np.nan, np.nan, fold,
                                    module_number, bsr, module_size]])
        else:

            row = pd.DataFrame(columns = list(svm_results.columns),
                        data = [[dataset_name, algorithm, center_method,
                                    data_dimension, center_dimension, fold,
                                    module_number, bsr, module_size]])

        svm_results = svm_results.append(row, ignore_index = True)

        module_number +=1

svm_results.head()

## Compute the relative gain in BSR.
Store in `svm_better`

In [ ]:
svm_better = pd.DataFrame(columns = ['Relative Gain', 'Fold', 'Prototype', 'Data Dimension', 'Center Dimension', 'Data Set'])

wgcna_query = (svm_results["Algorithm"] == 'WGCNA') &\
              (svm_results["Fold"] == fold) &\
              (svm_results["Data Set"] == dataset_name)
if np.sum(wgcna_query) > 0:
    wgcna_total = svm_results[wgcna_query]["BSR"].sum()
    eg_query = (svm_results["Algorithm"] == 'LBG') &\
            (svm_results["Central Prototype"] == center_method) &\
            (svm_results["Data Dimension"] == data_dimension) &\
            (svm_results["Center Dimension"] == center_dimension) &\
            (svm_results["Fold"] == fold) &\
            (svm_results["Data Set"] == dataset_name)                           
    eigengene_total = svm_results[eg_query]["BSR"].sum()
    if np.sum(eg_query) > 0:
        relative_gain = (eigengene_total/(wgcna_total+.0001))-1
        row = pd.DataFrame(columns = svm_better.columns,
                        data = [[relative_gain, fold, center_method, data_dimension, center_dimension, dataset_name]])
        svm_better = svm_better.append(row, ignore_index = True)
svm_better

## Plot Results

In [ ]:
%pip install nbformat==4.2.0

In [ ]:
px.box(svm_results, x = 'Algorithm', y = 'Size', color = 'Algorithm', points= 'all', title = 'Module Sizes')

In [ ]:
px.box(svm_results, x = 'Algorithm', y = 'BSR', color = 'Algorithm', points= 'all', title = 'Classification BSR')

In [ ]:
px.box(go_results, x = 'Algorithm', y = 'GO Significance', color = 'Algorithm', points= 'all', title = 'GO Significance')

In [ ]:
svm_better['Statistic'] = 'Classification BSR'
go_better['Statistic'] = 'GO Significance'
px.bar(pd.concat([svm_better, go_better]), y = 'Relative Gain', x = 'Statistic', 
       title = f'Relative gain for LBG modules over WGNCA modules <br>Module representative: {center_method} <br>Center dimension: {center_dimension}, Data dimension: {data_dimension}',
       barmode = 'group')